# Data Preperation

1. [Read all the files](#Read-all-the-files)
2. [Concat into a single pandas object](#Concat-into-a-single-pandas-object)
    * Remove duplicates
    * Drop NAs for job title
3. [Clean Job Titles](#Clean-Job-Titles)
4. [Save as a pickle](#Save-as-a-pickle)
5. [Example how to use pickle](#Example-how-to-use-pickle)

## Read all the files

In [1]:
import os

reviews_path = "../data/Database/"

files = [ reviews_path + i for i in os.listdir(reviews_path) if i.endswith("csv")]
display(files)

['../data/Database/accounting.csv',
 '../data/Database/automotive.csv',
 '../data/Database/computer_software.csv',
 '../data/Database/construction.csv',
 '../data/Database/education_management.csv',
 '../data/Database/financial.csv',
 '../data/Database/higher_ed.csv',
 '../data/Database/hospital_care.csv',
 '../data/Database/IT_services.csv',
 '../data/Database/retail.csv']

## Concat into a single pandas object

In [2]:
import pandas as pd

li = []

#Loading in first 2 files
for f in files:
    data = pd.read_csv(f)
    data["industry"] = f.split("/")[3][:-4]
    li.append(data)
    
reviews = pd.concat(li, axis=0, ignore_index=True)

print("Total Records", reviews.shape)

Total Records (2587686, 14)


Remove unwanted rows

In [3]:
reviews_ = reviews.copy()
reviews_ = reviews_.drop(columns=["Unnamed: 0"])

reviews_ = reviews_.drop_duplicates()

indices = reviews_.loc[reviews_.job_title.isna(),:].index
reviews_ = reviews_.drop(indices, axis=0)

reviews = reviews_
reviews_ = None #clear it

print("Job Titles NAs",reviews["job_title"].isna().sum())
print("Total Records", reviews.shape)
display( reviews.head() )

Job Titles NAs 0
Total Records (2350002, 13)


,company_name,review_title,job_title,employee_status,location,date,review,pros,cons,rating,yes_helpful,no_helpful,industry
0,EY,Wonderful place to work,Senior Executive Assistant,Former Employee,"Houston, TX",2020-03-19,This company goes above and beyond to provide ...,NaN,NaN,4.0,0,0,accounting
1,EY,People First,Senior Business Analyst,Former Employee,"New York, NY",2020-04-04,A place where you definitely will learn about ...,NaN,NaN,5.0,0,0,accounting
2,EY,I Enjoyed My Short Stay At EY!,Program Analyst,Former Employee,"Alpharetta, GA",2020-04-04,"This company wasn't perfect, but it's about as...","I love Work Life Balance! Also, they treat you...",Politics,4.0,0,0,accounting
3,EY,"Fun workplace, supportive team",Consultant,Former Employee,"Gurgaon, Haryana",2020-04-03,I have been appointed for busy season so we ha...,"Cab facilities, Flexibility for WFH, Supportiv...",team unity,4.0,0,0,accounting
4,EY,Great company to work for,Executive Assistant,Former Employee,"Canary Wharf, Greater London",2020-04-01,I wish I never resigned from EY - my massive m...,NaN,NaN,5.0,0,0,accounting


## Clean Job Titles

In [4]:
import string
import re

replacement_words = {
    "it":"technology"
    ,"sr":"senior"
    ,"qa":"quality"
    ,"sr": "senior"
    ,"jr": "junior"
}

translator = str.maketrans(string.punctuation, ' '*len(string.punctuation)) #map punctuation to space
def cleanTitle(title):
    title = str(title).translate(translator)
    title = re.sub(' +', ' ',title)
    title = title.lower()
    title_split = title.split(" ")
    for key, value in replacement_words.items():
        title_split = [ value if key == word else word for word in title_split ]
    return " ".join(title_split)

reviews["clean_job_title"] = reviews.job_title.apply(cleanTitle)


display( reviews[["job_title","clean_job_title"]].head() )

,job_title,clean_job_title
0,Senior Executive Assistant,senior executive assistant
1,Senior Business Analyst,senior business analyst
2,Program Analyst,program analyst
3,Consultant,consultant
4,Executive Assistant,executive assistant


## Save as a pickle

In [5]:
import pickle
import pathlib as Path

with open("../data/all_reviews.pkl","wb") as f:
    pickle.dump(reviews,f)

## Example how to use pickle

In [6]:
import pandas as pd
import pickle

with open("../data/all_reviews.pkl","rb") as f:
    reviews = pickle.load(f)
    
display(reviews.head())

,company_name,review_title,job_title,employee_status,location,date,review,pros,cons,rating,yes_helpful,no_helpful,industry,clean_job_title
0,EY,Wonderful place to work,Senior Executive Assistant,Former Employee,"Houston, TX",2020-03-19,This company goes above and beyond to provide ...,NaN,NaN,4.0,0,0,accounting,senior executive assistant
1,EY,People First,Senior Business Analyst,Former Employee,"New York, NY",2020-04-04,A place where you definitely will learn about ...,NaN,NaN,5.0,0,0,accounting,senior business analyst
2,EY,I Enjoyed My Short Stay At EY!,Program Analyst,Former Employee,"Alpharetta, GA",2020-04-04,"This company wasn't perfect, but it's about as...","I love Work Life Balance! Also, they treat you...",Politics,4.0,0,0,accounting,program analyst
3,EY,"Fun workplace, supportive team",Consultant,Former Employee,"Gurgaon, Haryana",2020-04-03,I have been appointed for busy season so we ha...,"Cab facilities, Flexibility for WFH, Supportiv...",team unity,4.0,0,0,accounting,consultant
4,EY,Great company to work for,Executive Assistant,Former Employee,"Canary Wharf, Greater London",2020-04-01,I wish I never resigned from EY - my massive m...,NaN,NaN,5.0,0,0,accounting,executive assistant
